<a href="https://colab.research.google.com/github/rafiq8k-moga/Gemini-AI-Subtitle-Translator/blob/main/Subtitle_ass_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Context Document
!pip install -q gradio google-generativeai
import gradio as gr
import google.generativeai as genai
import os
import time
from google.colab import userdata

# Konfigurasi API
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("API Key berhasil dikonfigurasi.")
except Exception as e:
    print(f"Gagal mengonfigurasi API Key. Error: {e}")

def parse_ass_file(file_path):
    """Membaca file .ass dan mengekstrak semua teks dialog."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            file_content = f.read()
    except Exception:
        with open(file_path, 'r', encoding='latin-1') as f:
            file_content = f.read()

    dialogue_texts = []
    for line in file_content.splitlines():
        if line.startswith('Dialogue:'):
            parts = line.split(',', 9)
            if len(parts) == 10:
                dialogue_texts.append(parts[9])
    return "\n".join(dialogue_texts)

def generate_context(current_ass_file, prev_context_file, model_choice, custom_instruction):
    logs = []
    try:
        # Pilih model
        model_map = {
            "Gemini 2.0 Flash": "gemini-2.0-flash",
            "Gemini 2.5 Flash": "gemini-2.5-flash"
        }
        model_name = model_map.get(model_choice, "gemini-2.0-flash")

        # 1. Ekstrak dialog
        logs.append("▶ Mengekstrak dialog dari file .ass...")
        current_dialogue = parse_ass_file(current_ass_file.name)
        if not current_dialogue:
            yield "❌ Tidak ada dialog yang ditemukan.", None, "\n".join(logs)
            return

        # 2. Baca konteks sebelumnya
        previous_context = "Tidak ada konteks dari episode sebelumnya."
        if prev_context_file is not None:
            logs.append("▶ Membaca file konteks sebelumnya...")
            with open(prev_context_file.name, 'r', encoding='utf-8') as f:
                previous_context = f.read()

        # 3. Buat prompt
        logs.append("▶ Membuat prompt...")
        prompt = f"""
        Anda adalah seorang AI asisten penulis skenario yang ahli dalam meringkas cerita dari dialog.
        Tugas Anda adalah membuat ringkasan konteks (context summary) yang padat dan informatif dari dialog sebuah episode. Ringkasan ini akan digunakan oleh AI lain untuk memastikan terjemahan episode selanjutnya akurat dan konsisten.

        IKUTI ATURAN-ATURAN INI:
        1.  Baca dan pahami konteks dari episode sebelumnya.
        2.  Baca seluruh dialog dari episode saat ini.
        3.  Buat ringkasan BARU berdasarkan dialog episode saat ini, sambil tetap menjaga konsistensi dengan konteks sebelumnya.

        Ringkasan HARUS mencakup:
        - Poin-poin plot utama yang terjadi di episode ini.
        - Nama semua karakter penting yang muncul dan tindakan kunci mereka.
        - Istilah, nama tempat, atau benda khusus yang baru diperkenalkan.
        - Hubungan antar karakter yang berkembang atau berubah.
        - Nada cerita atau cliffhanger di akhir episode.
        - {custom_instruction}

        ---
        KONTEKS DARI EPISODE SEBELUMNYA:
        ---
        {previous_context}
        ---
        DIALOG DARI EPISODE SAAT INI:
        ---
        {current_dialogue}
        ---
        RINGKASAN KONTEKS BARU UNTUK EPISODE SAAT INI:
        """

        # 4. Panggil Gemini API dengan streaming
        logs.append("▶ Menghubungi Gemini AI (streaming)...")
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(prompt, stream=True)

        generated_text = ""
        for chunk in response:
            if chunk.text:
                generated_text += chunk.text
                yield generated_text, None, "\n".join(logs)

        # 5. Simpan ke file .txt
        logs.append("▶ Menyimpan hasil ke file...")
        output_filename = f"context_{os.path.basename(current_ass_file.name).replace('.ass', '.txt')}"
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(generated_text)

        logs.append("🎉 Selesai! File siap diunduh.")
        yield generated_text, output_filename, "\n".join(logs)

    except Exception as e:
        logs.append(f"❌ Error fatal: {str(e)}")
        yield "", None, "\n".join(logs)

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI Context Maker for Subtitles (Streaming)")
    gr.Markdown("Buat ringkasan konteks dari file `.ass` untuk meningkatkan akurasi terjemahan di episode selanjutnya.")

    with gr.Row():
        with gr.Column(scale=2):
            ass_input = gr.File(label="1. Unggah File .ass Episode Saat Ini", file_types=[".ass"])
            prev_context_input = gr.File(label="2. (Opsional) Unggah Konteks Episode Sebelumnya", file_types=[".txt"])
            model_input = gr.Dropdown(
                label="3. Pilih Model AI",
                choices=["Gemini 2.0 Flash", "Gemini 2.5 Flash"],
                value="Gemini 2.0 Flash"
            )
            instruction_input = gr.Textbox(
                label="4. Instruksi Tambahan (Opsional)",
                lines=3,
                placeholder="Contoh: Fokus pada perkembangan hubungan antara Kirito dan Asuna."
            )
            submit_btn = gr.Button("Buat Konteks", variant="primary")

        with gr.Column(scale=3):
            context_output = gr.Textbox(
                label="Konteks yang Dihasilkan (Streaming)",
                lines=25,
                interactive=True,
                placeholder="AI akan mulai menulis ringkasan di sini..."
            )
            download_output = gr.File(label="Unduh Konteks (.txt)")
            log_output = gr.Textbox(
                label="Log Proses",
                lines=10,
                interactive=False
            )

    submit_btn.click(
        fn=generate_context,
        inputs=[ass_input, prev_context_input, model_input, instruction_input],
        outputs=[context_output, download_output, log_output]
    )

demo.launch(share=True, debug=True)


In [ ]:
# @title Translator
!pip install -q gradio google-generativeai

import gradio as gr
import google.generativeai as genai
import os
import json
import time
from google.colab import userdata

# Ambil API key dari Colab Secrets
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("API Key berhasil dikonfigurasi.")
except Exception as e:
    print(f"Gagal mengonfigurasi API Key. Pastikan Anda sudah menambahkannya di Secrets. Error: {e}")

# --- FUNGSI-FUNGSI HELPER (TIDAK PERLU DIUBAH) ---

def parse_ass_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()

    header_lines, dialogue_data, other_event_lines = [], [], []
    in_events_section = False
    lines = file_content.splitlines()
    for line in lines:
        if line.strip() == '[Events]': in_events_section = True
        if not in_events_section: header_lines.append(line)
        else:
            if line.startswith('Dialogue:'):
                parts = line.split(',', 9)
                if len(parts) == 10:
                    dialogue_data.append({'prefix': ','.join(parts[:9]) + ',', 'text': parts[9]})
            else: other_event_lines.append(line)
    header_lines.append('[Events]')
    return header_lines, dialogue_data, other_event_lines

def translate_batch(batch_data, model_name, source_lang, target_lang, context, instruction):
    model = genai.GenerativeModel(model_name)
    prompt = f"""**Role:** You are an expert and professional subtitle translator.**Task:** Translate the 'text' field for each object in the following JSON array from {source_lang} to {target_lang}.**Context:**\n{context}\n\n**Specific Instructions:**\n{instruction}\n\n**Crucial Rule:** Your output MUST be ONLY the JSON array, perfectly structured, with the translated text added in a new "translated_text" field.**Input JSON:**\n{json.dumps(batch_data, indent=2)}\n\n**Output JSON:**"""
    try:
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '').strip()
        return json.loads(cleaned_response)
    except Exception as e:
        print(f"Error saat memanggil API: {e}")
        return None

def rebuild_ass_file(header_lines, dialogue_data, translated_dialogues, other_event_lines):
    translated_map = {item['id']: item.get('translated_text', '') for item in translated_dialogues}
    final_content_lines = header_lines[:]

    reconstructed_events = []
    # Gabungkan semua event non-dialogue terlebih dahulu
    for line in other_event_lines:
        reconstructed_events.append(line)

    # Kemudian tambahkan semua dialog yang sudah diterjemahkan
    for i, dialog in enumerate(dialogue_data):
        translated_text = translated_map.get(i, dialog['text'])
        reconstructed_events.append(dialog['prefix'] + translated_text)

    final_content_lines.extend(reconstructed_events)
    return '\n'.join(final_content_lines)

def translate_process(subtitle_file, src_lang, tgt_lang, model_choice, context, instruction):
    # Mapping nama model di UI ke ID model API
    model_map = {
        "2.0 Flash": "gemini-2.0-flash",
        "2.5 Pro": "gemini-2.5-pro",
        "2.5 Flash": "gemini-2.5-flash"
    }
    model_name = model_map.get(model_choice, "gemini-2.0-flash-lite")

    log_history = "▶ Memulai proses...\n"
    yield {log_output: log_history, download_output: None}

    try:
        # Gradio memberikan objek file sementara, kita gunakan namanya (path)
        file_path = subtitle_file.name

        log_history += f"▶ Membaca file '{os.path.basename(file_path)}'...\n"
        yield {log_output: log_history}

        header_lines, dialogue_data, other_event_lines = parse_ass_file(file_path)
        num_dialogues = len(dialogue_data)

        log_history += f"✔ Ditemukan {num_dialogues} baris dialog.\n"
        yield {log_output: log_history}

        if num_dialogues == 0:
            log_history += "⚠ Tidak ada dialog untuk diterjemahkan. Proses selesai."
            yield {log_output: log_history}
            return

        batch_size = 150
        all_translated_dialogues = []
        num_batches = -(-num_dialogues // batch_size)

        for i in range(0, num_dialogues, batch_size):
            batch_num = (i // batch_size) + 1
            log_history += f"✈ Menerjemahkan batch {batch_num}/{num_batches}...\n"
            yield {log_output: log_history}

            current_batch_list = dialogue_data[i:i + batch_size]
            batch_to_translate = [{"id": i + j, "text": dialog['text']} for j, dialog in enumerate(current_batch_list)]

            translated_batch = translate_batch(batch_to_translate, model_name, src_lang, tgt_lang, context, instruction)

            if translated_batch is None:
                log_history += f"❌ Gagal menerjemahkan batch {batch_num}. Proses berhenti.\n"
                yield {log_output: log_history}
                return

            all_translated_dialogues.extend(translated_batch)
            log_history += f"✔ Batch {batch_num} selesai.\n"
            yield {log_output: log_history}
            time.sleep(1)

        log_history += "⚙ Semua batch selesai. Merekonstruksi file subtitle...\n"
        yield {log_output: log_history}

        final_ass_content = rebuild_ass_file(header_lines, dialogue_data, all_translated_dialogues, other_event_lines)

        output_filename = f"translated_{os.path.basename(file_path)}"
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(final_ass_content)

        log_history += f"🎉 Sukses! File hasil terjemahan '{output_filename}' siap diunduh.\n"
        yield {log_output: log_history, download_output: output_filename}

    except Exception as e:
        log_history += f"❌ Terjadi error fatal: {str(e)}\n"
        yield {log_output: log_history}

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI Subtitle Translator (Gradio & Gemini)")

    with gr.Row():
        with gr.Column(scale=2):
            # --- INPUTS ---
            file_input = gr.File(label="1. Unggah File Subtitle (.ass)", file_types=[".ass"])

            with gr.Row():
                source_lang_input = gr.Textbox(label="2. Dari Bahasa", value="English")
                target_lang_input = gr.Textbox(label="3. Ke Bahasa", value="Indonesian")

            model_input = gr.Dropdown(
                label="4. Pilih Model AI",
                choices=["2.0 Flash", "2.5 Pro", "2.5 Flash"],
                value="2.0 Flash"
            )

            context_input = gr.Textbox(
                label="5. Konteks (Opsional)",
                lines=3,
                placeholder="Contoh: Nama karakter adalah 'Ardi'. Nama jurus adalah 'Pukulan Seribu Bayang'."
            )

            instruction_input = gr.Textbox(
                label="6. Instruksi Khusus (Opsional)",
                lines=3,
                placeholder="Contoh: Gunakan gaya bahasa yang santai dan cocok untuk remaja."
            )

            submit_btn = gr.Button("Mulai Terjemahkan", variant="primary")

        with gr.Column(scale=3):
            # --- OUTPUTS ---
            download_output = gr.File(label="Unduh Hasil Terjemahan")
            log_output = gr.Markdown(label="Live Log", value="Menunggu proses...")

    # Menghubungkan tombol ke fungsi
    submit_btn.click(
        fn=translate_process,
        inputs=[file_input, source_lang_input, target_lang_input, model_input, context_input, instruction_input],
        outputs=[log_output, download_output]
    )

# Jalankan aplikasi dengan link publik (share=True)
demo.launch(share=True, debug=True)

# AI Subtitle Translator (.ass) via Gemini API


Proyek ini adalah sebuah *tool* canggih untuk menerjemahkan file subtitle berformat Advanced SubStation Alpha (`.ass`) secara otomatis menggunakan kekuatan model AI generatif dari Google, yaitu Gemini.

Merupakan evolusi signifikan dari proyek sebelumnya, [`Gemini-TXT-Translator`](https://www.google.com/search?q=%5Bhttps://github.com/rafiq8k-moga/Gemini-TXT-Translator%5D\(https://github.com/rafiq8k-moga/Gemini-TXT-Translator\)), tool ini dirancang untuk mengatasi kelemahan utama dari versi lama. Sebelumnya di `Gemini-TXT-Translator`, proses terjemahan menggunakan sistem per baris pada file `.txt` biasa, sebuah proses yang bisa sangat melelahkan dan menghilangkan semua konteks format subtitle.

Aplikasi baru ini, yang dibangun dengan Gradio dan dirancang untuk berjalan di Google Colab, menghadirkan alur kerja yang cerdas, efisien, dan jauh lebih ramah pengguna.

## Kelebihan & Fitur Utama

Tool ini dibangun untuk merevolusi proses fansubbing dengan AI, berikut adalah keunggulannya:

  * **Pemahaman Format `.ass` Cerdas**: Tool ini mem-parsing file `.ass` dan hanya mengambil teks dialog untuk diterjemahkan, **menjaga semua metadata penting** seperti timing, style, positioning, dan efek tetap utuh.
  * **Konteks Episode**: Anda dapat membuat **ringkasan konteks dari episode sebelumnya** sehingga AI lebih konsisten dalam menerjemahkan episode selanjutnya.
  * **Terjemahan Batch (Kelompok)**: Proses terjemahan tidak lagi dilakukan baris per baris. Dialog dikirim dalam kelompok (misalnya, 100 baris per permintaan), membuatnya **lebih cepat dan efisien**.
  * **Antarmuka Web Interaktif (Gradio)**: Semua interaksi dilakukan melalui antarmuka web yang bersih langsung di Colab.
  * **Live Logging Real-time**: Sidebar "Live Log" menampilkan update status setiap langkah, mulai dari membaca file hingga batch selesai diterjemahkan.
  * **Kontrol Penuh atas AI**: Anda bisa memberikan **konteks** (nama karakter, istilah khusus) dan **instruksi tambahan** untuk hasil terjemahan lebih akurat.
  * **Portabel dan Tanpa Setup**: Dijalankan sepenuhnya di **Google Colab**, tanpa perlu instalasi Python atau library tambahan.

## Panduan Penggunaan

Ikuti langkah-langkah sederhana ini untuk memulai.

### Langkah 0: (Penting) Siapkan File Subtitle Anda

File `.ass` seringkali berisi baris-baris kompleks yang bukan dialog lisan, seperti template karaoke (KFX), motion graphic, atau *shape*.  

Parser tool ini **hanya akan menerjemahkan baris yang diawali `Dialogue:`**, baris lain akan tetap ada di file hasil akhir.  

Jika ada baris `Dialogue:` yang tidak ingin diterjemahkan, bisa diabaikan manual di editor subtitle seperti **Aegisub**.

### Langkah 1: Jalankan Cell Context

Sebelum menerjemahkan, **jalankan cell Context** untuk membuat ringkasan konteks dari episode sebelumnya.  
Ini penting agar AI memahami alur cerita, karakter, dan istilah khusus sebelum menerjemahkan episode baru.  

1. Unggah file `.ass` episode saat ini.
2. Opsional: unggah file `.txt` konteks episode sebelumnya.
3. Pilih model AI.
4. Isi instruksi tambahan (opsional).
5. Klik **Buat Konteks**. Hasil ringkasan akan muncul dan bisa diunduh.

### Langkah 2: Jalankan Cell Translator

Setelah konteks dibuat, jalankan cell Translator untuk menerjemahkan episode.  

1. Unggah file `.ass` episode yang sama.
2. Pilih bahasa sumber dan target.
3. Pilih model AI.
4. Opsional: masukkan ringkasan konteks dari Langkah 1.
5. Klik **Mulai Terjemahkan**.
6. Saksikan proses live di sidebar log dan unduh file `.ass` hasil terjemahan setelah selesai.

## Perbandingan dengan Versi Lama (`Gemini-TXT-Translator`)

| Fitur                 | Gemini-TXT-Translator (Lama)                                             | AI Subtitle Translator (Baru)                                                              |
| :-------------------- | :----------------------------------------------------------------------- | :----------------------------------------------------------------------------------------- |
| **Proses Kerja** | Manual: Ekstrak dialog ke `.txt`, terjemahkan, lalu masukkan kembali.      | **Otomatis**: Unggah file `.ass`, terjemahkan, dan unduh file `.ass` yang sudah jadi.      |
| **Konteks Subtitle** | Semua timing, style, dan efek **hilang** saat diekstrak ke `.txt`.         | Semua timing, style, dan efek **100% terjaga**. Bisa menambahkan **konteks episode**.     |
| **Efisiensi** | Menerjemahkan **baris per baris**, lambat dan banyak panggilan API. | Terjemahkan dalam **batch**, lebih cepat dan efisien.                                       |
| **Feedback Proses** | Tidak ada. | **Live Log**: Memberikan update status untuk setiap langkah.                               |
| **Antarmuka** | Tidak ada. | **Antarmuka Web Interaktif** dari Gradio.                                                  |

## Teknologi yang Digunakan

  * **Python**
  * **Google Generative AI (Gemini API)**
  * **Gradio**
  * **Google Colab**
